In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [2]:
# load the data and word_index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key,value in word_index.items()}

In [3]:
# load the pretrained model with relu activation
model = load_model("simple_rnn_imdb.h5")

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [6]:
model.get_weights()

[array([[ 0.5019357 , -0.9150492 , -0.5859587 , ...,  0.83336323,
          0.925386  , -0.62507755],
        [ 0.04397861, -0.14415318, -0.07431584, ...,  0.12815274,
          0.10602982, -0.12596993],
        [ 0.12683503, -0.25024068, -0.18173803, ...,  0.15903258,
          0.11838492, -0.19107702],
        ...,
        [-0.04343547, -0.13214731,  0.06257723, ...,  0.03534196,
         -0.01856829, -0.09919417],
        [ 0.00798562, -0.11254667, -0.07979388, ...,  0.11180449,
          0.07067005, -0.04055928],
        [-0.06128032, -0.09545849,  0.01258173, ...,  0.12726995,
          0.04761112, -0.11317984]], dtype=float32),
 array([[ 0.02337438, -0.16957225,  0.12905103, ..., -0.00584007,
          0.07849209, -0.21712855],
        [-0.10210627,  0.05366961, -0.13613398, ...,  0.17952688,
          0.1209873 , -0.04689147],
        [-0.1054647 , -0.0671562 , -0.05262559, ..., -0.00325189,
         -0.00371112, -0.06254591],
        ...,
        [ 0.08099896, -0.1805056 ,  0.0

In [7]:
# create a function to decode the review
def decode_review(encoded_review):
    return ''.join([reverse_word_index.get(i - 3 , "?") for i in encoded_review])

# function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review] , maxlen = 500)
    return padded_review

In [10]:
# Prediction function
def predict_sentiment(review):
    preprocessed_input = preprocess_text(review)
    prediction = model.predict(preprocessed_input)
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [16]:
eg_review = "Boring and predictable. Fell asleep halfway through."

sentiment, prediction = predict_sentiment(eg_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [17]:
print(f"Review: {eg_review}" "\n"
      f"Sentiment: {sentiment}" "\n"
      f"Prediction score: {prediction}")

Review: Boring and predictable. Fell asleep halfway through.
Sentiment: Negative
Prediction score: 0.00029594358056783676
